In [ ]:
import json, subprocess, re, pyodbc, os, time, random
from sqlalchemy import func, create_engine
from bs4 import BeautifulSoup 
import numpy as np # linear algebra 
from config import mongoConn, connstr
import nltk
from nltk.corpus import stopwords
sw=stopwords.words('english')
import pandas as pd
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager 
import datetime as dt
from datetime import date, timedelta

In [ ]:
executable_path = {'executable_path': ChromeDriverManager().install()}

In [ ]:
with open('C:\\bom\\appsettings.json', encoding='utf-8-sig') as f:
    config = json.load(f)
pw = ''
for s in config['contexts']:
    if 'jira' in s['name']:
        pw=re.search('password,(.*);c:submit;', s['conn']).group(1)   

In [ ]:
browser = Browser('chrome', **executable_path, headless=False)
browser.visit(f"https://dayman.cyber-balance.com/jira/login.jsp")
browser.find_by_css('#login-form-username').fill('tim.kopp')
browser.find_by_css('#login-form-password').fill(pw)
browser.find_by_css('#login-form-submit').click()
html=browser.html

In [ ]:

def weeknum(dayname):
    if dayname == 'Monday':   return 0
    if dayname == 'Tuesday':  return 1
    if dayname == 'Wednesday':return 2
    if dayname == 'Thursday': return 3
    if dayname == 'Friday':   return 4
    if dayname == 'Saturday': return 5
    if dayname == 'Sunday':   return 6
def alldays(year, whichDayYouWant):
    d = date(year, 1, 1)
    d += timedelta(days = (weeknum(whichDayYouWant) - d.weekday()) % 7)
    while d.year == year:
        yield d
        d += timedelta(days = 7)

#for d in alldays(2021,'Monday'):
    #print(d)

In [ ]:
yr = 2021

In [ ]:
df = pd.read_csv(r'C:\Users\Tim\Documents\LOGS\jira.csv')  
df['task'] = df['Issue key'] + ' ' + df['Summary'] 
df['date'] = pd.to_datetime(df['Created'] )
df['updated'] = pd.to_datetime(df['Updated'] )
df=df[['task','date', 'updated']] 
today = dt.datetime.today().strftime("%m-%d-%Y")  
curyear = pd.to_datetime(today).year 
dfs = []
lst = []  
for monday in alldays(yr,'Monday'):
    dy = pd.to_datetime(monday).day
    mo = pd.to_datetime(monday).month  
    for d in pd.date_range(start=monday, periods=5, freq='D'): 
        for t in range(900, 1800, 100):   
            if mo==12: 
                dfd = df.loc[ (df.updated > pd.to_datetime(f'{yr}-{mo}-01')) & (df.updated < pd.to_datetime(f'{yr}-{mo}-30'))  ] 
            else:
                dfd = df.loc[ (df.updated > pd.to_datetime(f'{yr}-{mo}-01')) & (df.updated < pd.to_datetime(f'{yr}-{mo+1}-01'))  ] 
            if len(dfd) > 1: 
                idx = random.randrange(len(dfd)-1) 
                lst.append({'date': re.search('(.*)\s.*'
                            , str(d)).group(1), 'time': str(t) 
                            , 'task': dfd.iloc[idx].task }) 
    if len(lst) > 1:  
        dfs.append({'date': str(monday), 'df': pd.DataFrame( lst )}) 
        pd.DataFrame(lst).to_csv(f'C:\\Users\\Tim\\Documents\\LOGS\\tasklog\\_{str(monday)}.csv', index=False)
    lst = []

In [ ]:
df_dates = pd.DataFrame(dfs).sort_values('date', ascending=False) 

In [ ]:
with pd.ExcelWriter(f'C:\\Users\\Tim\\Documents\\LOGS\\{yr}LOG.xlsx') as writer:    
    for i, r in df_dates.iterrows():
        r.df.to_excel(writer, sheet_name=f'{r.date}')  

In [ ]:
browser = Browser('chrome', **executable_path, headless=False)
browser.visit(f"https://dayman.cyber-balance.com/jira/login.jsp")
browser.find_by_css('#login-form-username').fill('tim.kopp')
browser.find_by_css('#login-form-password').fill(pw)
browser.find_by_css('#login-form-submit').click()
browser.visit(f"https://dayman.cyber-balance.com/jira/issues/?jql=assignee%20%3D%20currentUser()%20AND%20project%3DCyberScope")
time.sleep(4)
html=browser.html
df = pd.read_html( html )[0]  

In [ ]:
soup = BeautifulSoup(html, 'html.parser')
els = soup.select('.issue-table-container')    